In [22]:
import gzip
import json
import ijson
import re
import os
import sys
import numpy as np
import pandas as pd
import time
from functools import reduce

In [2]:
DIR = '/Users/calummcmeekin/Downloads'

In [7]:
def count_lines(filename):
    with gzip.open(filename, 'rb') as f:
        for i, l in enumerate(f):
            pass
    return i


goodreads_books.json.gz has 2360654 lines

In [4]:
def load_isbn_data(file_name, head=1000):
    count = 0
    
    bookID_to_ISBN = {}

    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1

            bookID_to_ISBN[d['book_id']] = d['isbn']
                
            if ((count % 100000) == 0):
                print ("Currently exported: {}".format(count))
                
    return bookID_to_ISBN

In [6]:
def load_review_data(file_name):
    
    count = 0
    
    reviews_df = pd.DataFrame()
    
    review_to_bookID = {}
    
    with gzip.open(file_name) as fin:
        for l in fin:
            
            
            if (count < 15739966/3):
                
                d = json.loads(l)
              
                #append row to the dataframe
                #reviews_df = reviews_df.append(review_to_bookID, ignore_index=True)

                review_to_bookID[d['review_text']] = d['book_id']

                if ((count % 100000) == 0):
                    print ("Currently exported: {:.2f}%".format(count/(15739966/3)*100))

            else:
                break

            count += 1

                
    return review_to_bookID

**Export ISBN to book IDs**

In [10]:
start = time.process_time()

print ("_____________EXPORTING BOOKS______________")
bookID_to_ISBN = load_isbn_data(os.path.join(DIR, 'goodreads_books.json.gz'), )

print("\nTIME: {}".format(time.process_time() - start))

_____________EXPORTING BOOKS______________
Currently exported: 100000
Currently exported: 200000
Currently exported: 300000
Currently exported: 400000
Currently exported: 500000
Currently exported: 600000
Currently exported: 700000
Currently exported: 800000
Currently exported: 900000
Currently exported: 1000000
Currently exported: 1100000
Currently exported: 1200000
Currently exported: 1300000
Currently exported: 1400000
Currently exported: 1500000
Currently exported: 1600000
Currently exported: 1700000
Currently exported: 1800000
Currently exported: 1900000
Currently exported: 2000000
Currently exported: 2100000
Currently exported: 2200000
Currently exported: 2300000

TIME: 163.202305


**Get number of lines in each file**

In [153]:
num_books = count_lines(os.path.join(DIR, 'goodreads_books.json.gz'))
num_reviews = count_lines(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))

print("There are {} books".format(num_books))
print("There are {} book reviews".format(num_reviews))

There are 2360654 books
There are 15739966 book reviews


**Export reviews**

In [7]:
start = time.process_time()

print ("_____________EXPORTING REVIEWS______________")
reviews = load_review_data(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))

print("\nTIME: {}".format(time.process_time() - start))

_____________EXPORTING REVIEWS______________
Currently exported: 0.00%
Currently exported: 1.91%
Currently exported: 3.81%
Currently exported: 5.72%
Currently exported: 7.62%
Currently exported: 9.53%
Currently exported: 11.44%
Currently exported: 13.34%
Currently exported: 15.25%
Currently exported: 17.15%
Currently exported: 19.06%
Currently exported: 20.97%
Currently exported: 22.87%
Currently exported: 24.78%
Currently exported: 26.68%
Currently exported: 28.59%
Currently exported: 30.50%
Currently exported: 32.40%
Currently exported: 34.31%
Currently exported: 36.21%
Currently exported: 38.12%
Currently exported: 40.03%
Currently exported: 41.93%
Currently exported: 43.84%
Currently exported: 45.74%
Currently exported: 47.65%
Currently exported: 49.56%
Currently exported: 51.46%
Currently exported: 53.37%
Currently exported: 55.27%
Currently exported: 57.18%
Currently exported: 59.09%
Currently exported: 60.99%
Currently exported: 62.90%
Currently exported: 64.80%
Currently export

Number of unique reviews: 2,080,190

In [ ]:
reviews_df = pd.read_json(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))

**Convert JSON objects to Pandas DataFrame**

In [ ]:
# ISBN dataframe
isbn_df = pd.DataFrame()

isbn_df['isbn'] = bookID_to_ISBN.keys()
isbn_df['book_id'] = bookID_to_ISBN.values()


In [ ]:
# Book_ID and reviews dataframe
reviews_df = pd.DataFrame()

reviews_df['review'] = reviews.keys()
reviews_df['book_id'] = reviews.values()

In [9]:
reviews_df_2 = pd.DataFrame(reviews_df.items(), columns=['review_text', 'book_id'])

**Pickle the dataframe for quick access**

In [ ]:
isbn_df.to_pickle("isbn_df.pkl")

In [14]:
reviews_df_2.to_pickle("reviews_df.pkl")

KeyboardInterrupt: 

**Read Pickled Dataframes**

In [16]:
isbn_df = pd.read_pickle("isbn_df.pkl")

reviews_df = pd.read_pickle("reviews_df.pkl")

In [23]:
mixed_df = reduce(lambda x,y: pd.merge(x,y, on='book_id', how='outer'), [isbn_df, reviews_df])

In [ ]:
mixed_df.head(10)

In [34]:
mixed_df_2 = mixed_df[mixed_df['review_text'].isnull() == False]

In [37]:
del isbn_df
del reviews_df

In [38]:
mixed_df_2.to_pickle("clean_reviews_df.pkl")

In [39]:
print (mixed_df_2.shape)

(5054025, 3)


In [27]:
data = {}

index_key = 0

for cur_review in reviews:
    
    cur_review_json = {}
    
    cur_book_id = reviews[cur_review]
    
    cur_review_json['isbn'] = bookID_to_ISBN[cur_book_id]
    cur_review_json['book_id'] = reviews[cur_review]
    cur_review_json['review_text'] = cur_review
    
    data[index_key] = cur_review_json
    
    index_key += 1
        
    if (index_key % 100000 == 0):
        print ("Currently exported: {:.2f}%".format(index_key/15107456*100))
    

NameError: name 'reviews' is not defined

In [25]:
with open('skeleton_review_data.json', 'w') as outfile:
    json.dump(data, outfile)

{0: {'isbn': '', 'book_id': '24375664', 'review_text': "Mind blowingly cool. Best science fiction I've read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. \n The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn't have thought the ETO would exist in book 1, and I wouldn't have thought people would get so over-confident in our primitive fleet's chances given you have to think that with superior science they would have weapons - and defenses - that would just be as rifles to arrows once were. \n But the moment when Luo Ji won as a wallfacer was just too cool. I may have actually done a fist pump. Though by the way, if the Dark Forest theory is right - and I see no reason why it wouldn't be - we as a society should probably stop broadcasting so much sig

**Load data JSON to check contents**

In [4]:
filename = "skeleton_review_data.json"

ValueError: chunksize can only be passed if lines=True